In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import math
import seaborn as sns
import random
import sklearn

### Randomly Sample 10% of clean_df.csv due to large file size

In [5]:
p = 0.10  # 10% of the lines
# keep the header, then take only 10% of lines
# if random from [0,1] interval is greater than 0.10 the row will be skipped
clean_df_small = pd.read_csv(
    'clean_df.csv',
    header=0,
    index_col=0,
    skiprows=lambda i: i>0 and random.random() > p
)

In [6]:
clean_df_small.shape

(126214, 1147)

In [7]:
clean_df_small.info(verbose=1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126214 entries, 2 to 1260153
Data columns (total 1147 columns):
loan_amnt                               int64
funded_amnt                             float64
term                                    object
int_rate                                float64
installment                             float64
emp_length                              float64
home_ownership                          object
annual_inc                              float64
verification_status                     object
loan_status                             float64
purpose                                 object
zip_code                                object
addr_state                              object
dti                                     float64
delinq_2yrs                             int64
earliest_cr_line                        float64
fico_range_low                          int64
fico_range_high                         int64
inq_last_6mths                      

In [8]:
clean_df_small.drop(columns=['home_ownership','purpose','zip_code',
                             'addr_state','sub_grade','emp_title_2', #drop OHE source columns
                             'grade','emp_title'],inplace=True) #ALSO, drop redundant columns that new OHE columns provide the info for

In [9]:
clean_df_small.head()

,loan_amnt,funded_amnt,term,int_rate,installment,emp_length,annual_inc,verification_status,loan_status,dti,...,emp_title_2_Technician,emp_title_2_Truck Driver,emp_title_2_Vice President,emp_title_2_driver,emp_title_2_manager,emp_title_2_owner,emp_title_2_sales,emp_title_2_supervisor,emp_title_2_teacher,emp_title_2_truck driver
2,15000,15000.0,36 months,0.1042,486.98,3.0,55000.0,Source Verified,1.0,17.48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26,5000,5000.0,36 months,0.0944,160.03,4.0,55000.0,Source Verified,0.0,28.63,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
30,4000,4000.0,36 months,0.1199,132.84,1.0,36000.0,Not Verified,1.0,21.03,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
51,4000,4000.0,36 months,0.1709,142.80,1.0,30000.0,Source Verified,1.0,19.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58,15000,15000.0,60 months,0.1262,338.39,1.0,114000.0,Not Verified,1.0,9.24,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
print(clean_df_small.columns.values.tolist())

['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'emp_length', 'annual_inc', 'verification_status', 'loan_status', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_r

#### Eliminate Highly Correlated Columns & Plot Correlation Matrix

In [11]:
#drop date columns before correlation analysis/feature reduction
clean_df_no_dates = clean_df_small.drop(columns=['issue_d','last_pymnt_d'])

In [12]:
type(clean_df_no_dates)

pandas.core.frame.DataFrame

In [ ]:
corr_matrix = clean_df_no_dates.corr().abs()
high_corr_var=np.where(clean_df_no_dates.corr().abs()>0.8)
high_corr_var=[(corr_matrix.index[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

In [13]:
def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    '''Sort and show the top absolute value correlation pairs'''
    au_corr = df.corr().abs()#.unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(by=au_corr,ascending=False)
    return au_corr

In [14]:
print("Top Absolute Correlations:")

print(get_top_abs_correlations(clean_df_no_dates, n=10))

Top Absolute Correlations:


KeyError: "[('emp_title_2_Server', 'zip_code_039xx')\n ('zip_code_921xx', 'zip_code_424xx') ('zip_code_429xx', 'zip_code_237xx')\n ... ('zip_code_936xx', 'zip_code_114xx')\n ('emp_title_2_teacher', 'zip_code_804xx')\n ('zip_code_684xx', 'zip_code_555xx')] not found in axis"

#### Train-Test Split

In [ ]:
#### Train-Test Split
#     '''
#     Split dataset into 'train' and 'test'.
#     '''
#     import numpy as np
#     train, test = np.split(df.sample(frac=1), [int(.7*len(df))])

#     '''
#     Split train/test into features/label numpy arrays.
#     '''
#     LABEL_COL = 'loan_status'
#     train_x = train.loc[:, train.columns != LABEL_COL].values
#     train_y = train.loc[:, train.columns == LABEL_COL].values.squeeze()
#     test_x = test.loc[:, test.columns != LABEL_COL].values
#     test_y = test.loc[:, test.columns == LABEL_COL].values.squeeze()

#     # Standardize data
#     from sklearn.preprocessing import StandardScaler
#     scaler = StandardScaler()
#     scaler.fit(train_x)
#     train_x = scaler.transform(train_x)
#     test_x = scaler.transform(test_x)
#     np.savez_compressed('dataset.npz', train_x, train_y, test_x, test_y)
#     return (train_x, train_y, test_x, test_y)

In [ ]:
#### Train-Test Split & Standard Scaling
#     '''
#     Split dataset into 'train' and 'test'.
#     '''
#     import numpy as np
#     train, test = np.split(df.sample(frac=1), [int(.7*len(df))])

#     '''
#     Split train/test into features/label numpy arrays.
#     '''
#     LABEL_COL = 'loan_status'
#     train_x = train.loc[:, train.columns != LABEL_COL].values
#     train_y = train.loc[:, train.columns == LABEL_COL].values.squeeze()
#     test_x = test.loc[:, test.columns != LABEL_COL].values
#     test_y = test.loc[:, test.columns == LABEL_COL].values.squeeze()

#### Standard Scaling

In [ ]:
#     # Standardize data
#     from sklearn.preprocessing import StandardScaler
#     scaler = StandardScaler()
#     scaler.fit(train_x)
#     train_x = scaler.transform(train_x)
#     test_x = scaler.transform(test_x)
#     np.savez_compressed('dataset.npz', train_x, train_y, test_x, test_y)
#     return (train_x, train_y, test_x, test_y)

## Classification Modeling (Fully Paid = 1, Charged-Off = 0)

#### Logistic Regression v1

## IRR Target Variable Calculation/Extrapolation

In [ ]:
#calculating NAR
lc_df[['total_rec_int','total_rec_late_fee','installment','collection_recovery_fee','out_prncp','loan_status']].head(5)

In [ ]:
lc_df['last_pymnt_d'].head()

In [ ]:
lc_df['last_payment_date'] = lc_df['last_pymnt_d'].str[:3]+'/'+'1'+'/'+lc_df['last_pymnt_d'].str[4:]
lc_df['issue_date'] = lc_df['issue_d'].str[:3]+'/'+'1'+'/'+lc_df['issue_d'].str[4:]

In [ ]:
lc_df['last_payment_date'] = lc_df['last_payment_date'].astype(str)
lc_df['issue_date'] = lc_df['issue_date'].astype(str)

In [ ]:
lc_df = lc_df[lc_df['last_payment_date'] != 'nan'].reset_index()

In [ ]:
lc_df['last_payment_date'][1319]

In [ ]:
from datetime import datetime

lc_df['last_payment_date_dt'] = lc_df['last_payment_date'].map(lambda x: datetime.strptime(x,'%b/%d/%Y'))
lc_df['issue_date_dt'] = lc_df['issue_date'].map(lambda x: datetime.strptime(x,'%b/%d/%Y'))

In [ ]:
lc_df['issue_date_dt'].tail()

In [ ]:
lc_df['days_btwn_funding_lastpayment'] = (lc_df['last_payment_date_dt'].dt.date -
                                          lc_df['issue_date_dt'].dt.date).dt.days

In [ ]:
#raw_lc_df['y_stanford'] = (raw_lc_df['total_pymnt']/raw_lc_df['funded_amnt'])-1

In [ ]:
lc_df[['addr_state','annual_inc','collection_recovery_fee','emp_title',
           'fico_range_high','fico_range_low','funded_amnt','grade','home_ownership',
           'int_rate','loan_amnt','loan_status','purpose','sub_grade','title','total_rec_int',
           'total_rec_late_fee','total_rec_prncp','zip_code','debt_settlement_flag','out_prncp',
           'collection_recovery_fee','days_btwn_funding_lastpayment','y_stanford']][lc_df['loan_status'] == 1].tail(5)

In [ ]:

# for i in range(50): 
#     if raw_lc_df.loc[i,'loan_status'] != 'Charged Off':
#         raw_lc_df.loc[i,'NAR_test'] = ((1+((((raw_lc_df['total_rec_int'][i] #(interest received
#                                     +raw_lc_df['total_rec_late_fee'][i] # + late fees received
#                                     -(0.01*raw_lc_df['installment'][i]) # - service fee paid
#                                     +((raw_lc_df['collection_recovery_fee'][i]/.4)*.6) # + collection fees received
#                                     - 0) # - 0 or out.principal
#                                  /(raw_lc_df['out_prncp'][i])) #ALL THE ABOVE divided by out.principal
#                                     *raw_lc_df['out_prncp'][i]) #FRACTION ABOVE times out.principal
#                                  / (raw_lc_df['out_prncp'][i])))**12)-1 #TERM ABOVE divided by out.principal, 
#                                                                     #& EVERYTHING to the power of 12 & ALL OF THAT minus 1
#     else: 
#         raw_lc_df.loc[i,'NAR_test'] = ((1+((((raw_lc_df['total_rec_int'][i] #(interest received
#                                     +raw_lc_df['total_rec_late_fee'][i] # + late fees received
#                                     -(0.01*raw_lc_df['installment'][i]) # - service fee paid
#                                     +((raw_lc_df['collection_recovery_fee'][i]/.4)*.6) # + collection fees received
#                                     - raw_lc_df['out_prncp'][i]) # - 0 or out.principal
#                                  /(raw_lc_df['out_prncp'][i])) #ALL THE ABOVE divided by out.principal
#                                     *raw_lc_df['out_prncp'][i]) #FRACTION ABOVE times out.principal
#                                  / (raw_lc_df['out_prncp'][i])))**12)-1 #TERM ABOVE divided by out.principal, 
#                                                                     #& EVERYTHING to the power of 12 & ALL OF THAT minus 1
#     print('row completed')

In [ ]:
lc_df.head(10)

In [ ]:
lc_df['loan_status'].value_counts()